# Bedingte Zuweisung einer neuen Zeile in Pandas

In [1]:
import numpy as np
import pandas as pd

## Ausgangslage

Gegeben sei ein Data Frame mit 100000 Torergebnissen:

In [2]:
spiele = pd.DataFrame( {
    'Heimteam': np.random.randint(10, size=100000), 'Auswaertsteam': np.random.randint(10, size=100000)
} )
spiele.head()

,Auswaertsteam,Heimteam
0,1,4
1,0,3
2,5,0
3,2,6
4,1,3


Ziel: Einführen einer neuen Zeile, die besagt, ob das Heimteam gewonnen (+1), verloren (-1) oder unentschieden (0) gespielt hat.

Dazu berechnen wir als erstes die Differenz zwischen den Torergebnissen.

In [3]:
spiele['differenz'] = spiele.Heimteam - spiele.Auswaertsteam
spiele.head()

,Auswaertsteam,Heimteam,differenz
0,1,4,3
1,0,3,3
2,5,0,-5
3,2,6,4
4,1,3,2


Wie es jetzt weitergeht, ist eigentlich klar: Alle negativen Werte sollen zu einer -1 werden, alle positiven zu einer +1 und alle Nullen zu einer 0.

Die Möglichkeiten dazu wollen wir im Folgenden kurz durchgehen:

## Ansatz -1: For Loop (Negativ, weil er so schlecht ist..)

Wir definieren eine Funktion `winornot` und wenden sie per for-Loop auf jede Zeile an:

In [4]:
def winornot( x ):
    if x > 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1

In [5]:
%%timeit
resultat = []
for i in range( spiele.shape[0] ):
    resultat.append( winornot( spiele['differenz'][i] ) )
spiele['outcome'] = resultat

1.22 s ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Ansatz 0 (Riccard's Idee):

Wir benutzen die `select`-Methode von numpy:

In [6]:
%%timeit
conditions = [ spiele['differenz']>0, spiele['differenz']<0, spiele['differenz']==0] 
choices = [1,-1,0]
spiele['outcome'] = np.select( conditions, choices, default='null')

16.5 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ansatz 0 ist schon über 100 mal schneller als Ansatz -1. Er ist aber nicht sofort lesbar und benötigt Kenntnisse über `np.select()`.

## Ansatz 1: Map von Pandas

Wir definieren eine Funktion, die für eine Differenz das entsprechende Ergebnis zurückgibt und benutzen `map`:

In [7]:
%%timeit
spiele['differenz'].map( winornot )

30.3 ms ± 566 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Ansatz 1 ist leicht verständlich und bleibt im Framework von Pandas, ist leider aber nur als halb so schnell wie Ansatz 0, weil er nicht vektorisiert ist.

## Ansatz 2: Indizierung mit Arrays aus logischen Werten

In [8]:
%%timeit
spiele['outcome'] = spiele['differenz'].copy()
spiele['outcome'][spiele['outcome'] > 0] = 1
spiele['outcome'][spiele['outcome'] < 0] = -1
spiele['outcome'][spiele['outcome'] == 0] = 0

8.48 ms ± 92.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ansatz 2 ist sehr leicht lesbar, bleibt im Pandas Framework und ist noch leicht schneller als Ansatz 0 und 1, da vektorisiert. Er nimmt aber mehrere Zeilen Code ein, was etwas viel ist für die einfache Operation. 

## Ansatz 3: Die Signumsfunktion von Numpy

Die mathematische Funktion `sign(x)` gibt 1 für ein positives Vorzeichen, -1 für ein negatives Vorzeichen und 0 für eine Null zurück. Also genau das, was wir suchen!

In [9]:
%%timeit
spiele['outcome'] = np.sign( spiele['differenz'] )

472 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Ansatz 3 ist definitiv der schnellste! Er passt schön in eine Zeile und ist über 2000 mal schneller als der naive `for`-Loop vom Anfang!

## Ansatz 4:

Man kann sich auch überlegen, wie eine Signumsfunktion zustandekommt. Man könnte zum Beispiel jede Zahl durch ihren Absolutbetrag teilen. Für die Null gibts da natürlich ein Problem, aber da Divisionen durch 0 in Pandas einfach ein 'NaN' ergeben, können wir diese einfach durch eine Null ersetzen:

In [10]:
%%timeit
(spiele['differenz'] / spiele['differenz'].abs()).fillna(0)

1.11 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Ansatz 4 ist etwas 'ugly', aber der zweitschnellste!

## Ansatz 5 (Roman):

In [11]:
%%timeit
spiele['outcome'] = np.where( spiele['differenz']==0, 0, np.where( spiele['differenz']>0, 1, -1 ) )

1.63 ms ± 90.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Dieser Ansatz ist in etwa 3x langsamer als der mit der Signumsfunktion (aber immer wesentlich schneller als die anderen) und meiner Meinung nach noch etwas weniger schnell lesbar als Ansätze 1,2 und 3.